# PRE-PROCESSING

In [ ]:
import numpy as np
import h5py
import numpy as np
from scipy.integrate import solve_ivp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
import torch
import matplotlib.pyplot as plt

In [ ]:
# Init pack

import numpy as np
import h5py
import numpy as np
from scipy.integrate import solve_ivp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
import torch
import matplotlib.pyplot as plt

psi = np.load('/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy')

# Pre-processing

lead = 1

trainN = 200000
valN = 50000
index = 63

psi = psi[:,1,:]

print(psi.shape)
mean_psi = np.mean(psi, axis=0, keepdims=True)
std_psi = np.std(psi, axis=0, keepdims=True)
psi = (psi - mean_psi) / std_psi

psi_train_input = torch.tensor(psi[0:trainN,:])
psi_train_label =  torch.tensor(psi[lead:trainN+lead,:])


# # Make sure input and label lengths match
# assert psi_train_input.shape[0] == psi_train_label.shape[0], "Input and label length mismatch"

# #shuffle and map indices\
# valid_indices = np.arange(0, trainN - lead)
# np.random.seed(42)  # Optional for reproducibility
# shuffled_indices = np.random.permutation(valid_indices)
# psi_train_input = torch.tensor(psi[shuffled_indices, :])
# psi_train_label = torch.tensor(psi[shuffled_indices + lead, :])


# t = shuffled_indices[0]
# print(torch.allclose(psi_train_input[0], torch.tensor(psi[t])))
# print(torch.allclose(psi_train_label[0], torch.tensor(psi[t + 1])))

psi_val_input = torch.tensor(psi[trainN:trainN+valN,:])
psi_val_label =  torch.tensor(psi[trainN+lead:trainN+valN+lead,:])

print(psi_train_input.shape)
print(psi_train_label.shape)
print(psi_val_input.shape)
print(psi_val_label.shape)
plt.plot(psi_train_input[0:200000,63])
plt.plot(psi_val_input[0:50000,63])

# ARCHITECTURE

In [ ]:
# Define the encoder (MLP)
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(75, 512)  # Input layer (2 + 2) -> Hidden layer (128)
        self.fc2 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc3 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc4 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc5 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc6 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc_mu = nn.Linear(512, latent_dim)  # Hidden layer (128) -> Latent space (2)
        self.fc_logvar = nn.Linear(512, latent_dim)  # Hidden layer (128) -> Log variance (2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Activation function for hidden layer
        x = torch.relu(self.fc2(x)) + x
        x = torch.relu(self.fc3(x)) + x
        x = torch.relu(self.fc4(x)) + x
        # x = torch.relu(self.fc5(x)) + x
        # x = torch.relu(self.fc6(x)) + x
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

# Define the decoder (MLP)
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim, condition_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim + condition_dim, 512)  # Input layer (2 + 2) -> Hidden layer (128)
        self.fc2 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc3 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc4 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc5 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc6 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc_output = nn.Linear(512, output_dim)  # Hidden layer (128) -> Output layer (2)

    def forward(self, z, condition):
        z = torch.cat((z, condition), dim=1)  # Concatenate latent vector and condition
        z = torch.relu(self.fc1(z))  # Activation function for hidden layer
        z = torch.relu(self.fc2(z)) + z
        z = torch.relu(self.fc3(z)) + z
        z = torch.relu(self.fc4(z)) + z
        # z = torch.relu(self.fc5(z)) + z
        # z = torch.relu(self.fc6(z)) + z
        output = self.fc_output(z)
        return output

# Define the VAE model
class ConditionalVAE(nn.Module):
    def __init__(self, latent_dim, output_dim, condition_dim):
        super(ConditionalVAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim, output_dim, condition_dim)

    def encode(self, x):
        mu, logvar = self.encoder(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z, condition):
        return self.decoder(z, condition)

    def forward(self, x, condition):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        output = self.decode(z, condition)
        return output, mu, logvar

output_dim = 75
latent_dim = 1024
condition_dim = 75
batch_size = 1024
num_epochs = 100

# ORIGINAL TRAINING + INFERENCE

In [ ]:
# Training

import datetime
import os
import math

def normalize_transition_time(slope_value, delta, transition_real):
    normalized = 1 - np.exp(-np.abs((slope_value - transition_real)) / delta)
    return normalized

def total_variation_distance(p, q):
    p = np.array(p)
    q = np.array(q)
    return 0.5 * np.sum(np.abs(p - q))

def calculate_transition_durations(y_values, upper_bound, lower_bound):
    times_between_transitions = []
    transition_start = None
    above_upper = False
    below_lower = False
    for i in range(1, len(y_values)):
        if y_values[i] < lower_bound:
            below_lower = True
            above_upper = False
        elif y_values[i] > upper_bound:
            if below_lower and transition_start is not None:
                times_between_transitions.append(i - transition_start)
                transition_start = None
            above_upper = True
            below_lower = False
        if below_lower and transition_start is None:
            transition_start = i
    return times_between_transitions

latent_dims = [1024]
latent_dim = 1024
kl_coefficients = [10]
num_cycles = 5
upper_bound = 53.8 / 2.8935
lower_bound = 7.41
level = 63

best_distance = float('inf')
tvd_list = []
transition_list = []
transition_list_unormalized = []

real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
real_data_1d = real_data[:, 1, level]
real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)
real_data_sorted = np.sort(real_durations)
transition_real = np.mean(real_data_sorted)
actual_hist, bin_edges = np.histogram(real_data[:, 1, level], bins=50, density=True)
print(f"Reference Real Data average_transition_time: {transition_real}")

tvds_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}
transitions_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}
transitions_normalized_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}
exp_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}

lat_folder = f"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_KL_TEST_at_{datetime.datetime.now()}"
os.makedirs(lat_folder)

for kl_coef in kl_coefficients:
    print(f"USING KL COEF OF {kl_coef}")
    best_models = []
    best_models_saved = []
    for cycle in range(0,num_cycles):
        tvd_cycle_list = []
        transition_cycle_list = []
        transition_cycle_list_unormalized = []
        
        # Initialize the model, optimizer, and loss function
        model = ConditionalVAE(latent_dim, output_dim, condition_dim)
        model = model.cuda()
        optimizer = optim.Adam(model.parameters(), lr=0.0001)
        num_epochs = 5

        folder = f"{lat_folder}/model_at_{cycle}_with_kl_{kl_coef}"
        os.makedirs(folder)
        subfolders = ['timeseries', 'expo_fit', '2D', 'summary']

        # Create each subdirectory inside the main folder
        for subfolder in subfolders:
            path = os.path.join(folder, subfolder)
            os.mkdir(path)
            print(f"Created subfolder: {path}")

        # from torchsummary import summary

        # summary(model, input_size = [(128, 1, 75), (128, 75)])
        # Train the model
        for epoch in range(num_epochs):
            model.train()
            for batch in range (0, trainN, batch_size):

                input_batch = psi_train_input[batch:batch + batch_size,:]
                label_batch = psi_train_label[batch:batch + batch_size,:]

                optimizer.zero_grad()

                output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())

                # Reconstruction loss
                reconstruction_loss = F.mse_loss(output, label_batch.float().cuda(), reduction="sum")
                # KL divergence loss
                kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
                
                # Total loss
                loss = reconstruction_loss + kl_coef * kl_loss

                loss.backward()
                optimizer.step()

            print(f'''Epoch {epoch+1}, 
                Reconstrunction Loss: {reconstruction_loss},
                KL Divergence Loss: {kl_loss}''')

            # Validation Loss
            for batch in range (0, valN, batch_size):

                input_batch = psi_val_input[batch:batch + batch_size,:]
                label_batch = psi_val_label[batch:batch + batch_size,:]
                
                output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())
                reconstruction_loss = F.mse_loss(output, label_batch.float().cuda(), reduction="sum")
                kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
                val_loss = reconstruction_loss + kl_coef * kl_loss # Experiment HIGHER coefficients
                # Print both reconstruction_loss and kl_loss

            print(f'''
                Validation Reconstrunction Loss: {reconstruction_loss},
                Validation KL Divergence Loss: {kl_loss}''')
            
            # Inference

            initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
            time_step = 300000
            z = torch.zeros([1,latent_dim])
            num_ens = 1
            pred = np.zeros ([time_step, 75, num_ens])

            for k in range (0, time_step):

                for ens in range (0, num_ens):
                    if (k ==0):

                        z = torch.randn_like(z)
                        print(z.shape, initial_cond.shape)
                        y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                        pred[k,:,ens] = y
                        y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                        initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

                    else:
                        select_ens = np.random.randint(0,num_ens,1)
                        z = torch.randn_like(z)
                        y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                        pred[k,:, ens] = y
                        y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                        initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            # Denormalize final preds
            print(std_psi[:, 63])
            pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

            # Denormalize test labels
            actual_values = psi_train_label[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
            print(actual_values)
            
            real_data_1d = real_data[:30000, 0, 63]  # Now shape is (309700,)
            predictions_1d = pred_mean[:, 63]  # shape (300000,)

            plt.figure(figsize=(20,8))
            plt.plot(pred_mean[0:30000,63],'r')
            plt.plot(real_data[0:30000, 0, 63])
            plt.grid(True)
            plt.title(f"Predictions vs Actual | Epoch {epoch}")
            save_path = os.path.join(folder, "timeseries")
            save_path = os.path.join(save_path, f"timeseries_plot_{epoch}.png")
            plt.savefig(save_path)
            plt.xlabel('Time Step')
            plt.ylabel('Zonal Wind Value')
            plt.legend(['Predictions', 'Actual'])
            plt.tight_layout()
            plt.show()
            
            # Function to calculate transition durations
            def calculate_transition_durations(y_values, upper_bound, lower_bound):
                times_between_transitions = []
                transition_start = None
                above_upper = False
                below_lower = False

                for i in range(1, len(y_values)):
                    if y_values[i] < lower_bound:  
                        below_lower = True
                        above_upper = False
                    elif y_values[i] > upper_bound:  
                        if below_lower and transition_start is not None:
                            times_between_transitions.append(i - transition_start)
                            transition_start = None  
                        above_upper = True
                        below_lower = False

                    if below_lower and transition_start is None:
                        transition_start = i

                return times_between_transitions
            
            predictions_1d = pred_mean[:, level, 0]
            pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)
            print(pred_durations)
            pred_hist, _ = np.histogram(predictions_1d, bins=bin_edges, density=True)

            # === PREDICTIONS CCDF AND FIT ===
            exponential_fit_pred = 0

            # if len(pred_durations) > 0:
            transition_pred = np.mean(pred_durations)
            transition_cycle_list.append(normalize_transition_time(transition_pred, 1000, transition_real))
            transition_cycle_list_unormalized.append(transition_pred)
            transitions_by_dim_cycle[kl_coef][cycle].append(transition_pred)
            transitions_normalized_by_dim_cycle[kl_coef][cycle].append(normalize_transition_time(transition_pred, 1000, transition_real))

            tvd = total_variation_distance(pred_hist, actual_hist)
            tvd_cycle_list.append(tvd)
            tvds_by_dim_cycle[kl_coef][cycle].append(tvd)

            distance = np.sqrt(tvd ** 2 + (normalize_transition_time(transition_pred, 1000, transition_real)) ** 2)

            print(f"Epoch {epoch+1}: TVD = {tvd:.6f}, Transition_Difference = {transition_pred-transition_real:.6f}, Combined Distance = {distance:.6f}")
            torch.save(model.state_dict(), f"{folder}/model_at_epoch{epoch}")

            if distance < best_distance:
                best_distance = distance
                torch.save(model.state_dict(), f"{folder}/best_model_combined_distance.pth")
                print("New best model saved based on TVD + transition difference distance.")
            # else:
            #     print("No transitions detected in predictions.")
            #     transitions_by_dim_cycle[kl_coef][cycle].append(np.nan)
            #     tvds_by_dim_cycle[kl_coef][cycle].append(np.nan)
            #     tvd_cycle_list.append(np.nan)
            #     transition_cycle_list.append(np.nan)
            #     transition_cycle_list_unormalized.append(np.nan)


            # Plot labels and formatting

            x_line_real = np.linspace(min(real_data_sorted), max(real_data_sorted), 100)
            exponential_fit_real = 1/np.mean(real_data_sorted)
            y_values_real = exponential_fit_real*x_line_real
            plt.plot(x_line_real, y_values_real, 'b-', label=f'Real Exp Fit (slope={exponential_fit_real:.4f})', linewidth=2)

            if len(pred_durations) > 0:
                x_line_pred = np.linspace(min(pred_durations), max(pred_durations), 100)
                exponential_fit_pred = 1/np.mean(pred_durations)
                y_values_pred = exponential_fit_pred*x_line_pred
                plt.plot(x_line_pred, y_values_pred, 'r-', label=f'Pred Exp Fit (slope={exponential_fit_pred:.4f})', linewidth=2)
                exp_by_dim_cycle[kl_coef][cycle].append(exponential_fit_pred)

            plt.xlabel('Time Duration (Steps)')
            plt.ylabel('Exponential Fit')
            plt.title('Exponential Fit of Time Between B->A and A->B Transitions (Exponential Fit)')
            plt.yscale("linear")  # y-axis log scale
            plt.xscale("linear")  # x-axis linear scale
            plt.grid()
            plt.legend()
            plt.tight_layout()
            save_path = os.path.join(folder, "expo_fit")
            save_path = os.path.join(save_path, f"expo_fit_plot_{epoch}.png")
            plt.savefig(save_path)
            plt.show()

            plt.figure(figsize=(12, 6))
            plt.plot(tvd, transition_pred, 'o-', label=f'Cycle {cycle}')
            plt.xlabel("Total Variation Distance")
            plt.ylabel("Average Transition Time")
            plt.axhline(y=transition_real, color='r', linestyle='--', label='Real Data')
            plt.ylim(0.1,2000)
            plt.xlim(0, 1)
            plt.title("TVD vs. Avg Transition Time per Epoch")
            plt.grid(True)
            save_path = os.path.join(folder, "2D")
            save_path = os.path.join(save_path, f"2D_plot_{epoch}.png")
            plt.savefig(save_path)
            plt.show()
            
            if epoch == num_epochs - 1:  # Last epoch of last cycle for this dimension
                # Plot TVDs with cycles overlapped
                plt.figure(figsize=(10, 6))
                for c in range(num_cycles):
                    plt.plot(tvds_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                
                plt.xlabel('Epoch within Cycle')
                plt.ylabel('Total Variation Distance')
                plt.title(f'TVD Progress (KL Coefficient={kl_coef})')
                plt.grid(True)
                plt.legend()
                save_path = os.path.join(folder, "summary")
                save_path = os.path.join(save_path, f"tvd_plot_all_cycles.png")
                plt.savefig(save_path)
                plt.show()
                
                # Plot Exponential Fits with cycles overlapped
                plt.figure(figsize=(10, 6))
                for c in range(num_cycles):
                    plt.plot(transitions_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                
                plt.axhline(y=transition_real, color='r', linestyle='--', label='Real Data')
                plt.xlabel('Epoch within Cycle')
                plt.ylabel('Average Transition Value')
                plt.ylim(0.1,2000)
                plt.title(f'Average Transition Progress (KL Coefficient={kl_coef})')
                plt.grid(True)
                plt.legend()
                save_path = os.path.join(folder, "summary")
                save_path = os.path.join(save_path, f"transition_plot_all_cycles.png")
                plt.savefig(save_path)
                plt.show()

                plt.figure(figsize=(10, 6))
                for c in range(num_cycles):
                    plt.plot(exp_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                
                plt.axhline(y=exponential_fit_real, color='r', linestyle='--', label='Real Data')
                plt.xlabel('Epoch within Cycle')
                plt.ylabel('Exponential Fit Value')
                plt.title(f'Exponential Fit Progress (KL Coefficient={kl_coef})')
                plt.grid(True)
                plt.legend()
                save_path = os.path.join(folder, "summary")
                save_path = os.path.join(save_path, f"exponential_fit_plot_all_cycles.png")
                plt.savefig(save_path)
                plt.show()
                
                # Plot TVD and 1/average transition time per epoch
                plt.figure(figsize=(12, 6))
                plt.scatter(tvd_cycle_list, transition_cycle_list, c=range(len(tvd_cycle_list)), cmap='viridis')
                plt.colorbar(label='Epoch')
                plt.xlabel("Total Variation Distance")
                plt.ylabel("Average Transition Time")
                plt.title("TVD vs. Avg Transition Time per Epoch")
                plt.grid(True)
                save_path = os.path.join(folder, "summary")
                save_path = os.path.join(save_path, f"tvd_vs_transition_plot_all_cycles.png")
                plt.savefig(save_path)
                plt.show()
                
                # If this is the last latent dimension, create master graphs with all dimensions
                if kl_coef == kl_coefficients[-1] and cycle == num_cycles - 1:
                    # Create a master folder for overlapping graphs
                        
                        # Plot TVDs with cycles overlapped
                    plt.figure(figsize=(10, 6))
                    for c in range(num_cycles):
                        plt.plot(tvds_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                    
                    plt.xlabel('Epoch within Cycle')
                    plt.ylabel('Total Variation Distance')
                    plt.title(f'TVD Progress (KL Coefficient={kl_coef})')
                    plt.grid(True)
                    plt.legend()
                    save_path = os.path.join(lat_folder, f"tvd_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()
                    
                    # Plot Exponential Fits with cycles overlapped
                    plt.figure(figsize=(10, 6))
                    for c in range(num_cycles):
                        plt.plot(transitions_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                    
                    plt.axhline(y=transition_real, color='r', linestyle='--', label='Real Data')
                    plt.xlabel('Epoch within Cycle')
                    plt.ylabel('Average Transition Value')
                    plt.ylim(0.1,2000)
                    plt.title(f'Average Transition Progress (KL Coefficient={kl_coef})')
                    plt.grid(True)
                    plt.legend()
                    save_path = os.path.join(lat_folder, f"transition_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()

                    plt.figure(figsize=(10, 6))
                    for c in range(num_cycles):
                        plt.plot(exp_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                    
                    plt.axhline(y=exponential_fit_real, color='r', linestyle='--', label='Real Data')
                    plt.xlabel('Epoch within Cycle')
                    plt.ylabel('Exponential Fit Value')
                    plt.title(f'Exponential Fit Progress (KL Coefficient={kl_coef})')
                    plt.grid(True)
                    plt.legend()
                    save_path = os.path.join(lat_folder, f"exponential_fit_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()
                    
                    # Plot TVD and 1/average transition time per epoch
                    plt.figure(figsize=(12, 6))
                    plt.scatter(tvd_cycle_list, transition_cycle_list, c=range(len(tvd_cycle_list)), cmap='viridis')
                    plt.colorbar(label='Epoch')
                    plt.xlabel("Total Variation Distance")
                    plt.ylabel("Average Transition Time")
                    plt.title("TVD vs. Avg Transition Time per Epoch")
                    plt.grid(True)
                    save_path = os.path.join(lat_folder, f"tvd_vs_transition_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()

            torch.save(model.state_dict(), f"{folder}/checkpoint_{epoch+1}")
            print(f"Model weights saved to {folder} with point {epoch+1}.")
        
            tvd_list.append(tvd_cycle_list)
            transition_list.append(transition_cycle_list)
            transition_list_unormalized.append(transition_cycle_list_unormalized)

        # Save the model after each cycle
        index = -1
        best_distance = float('inf')
        for i in range(0, num_cycles):
            distance = np.sqrt(tvds_by_dim_cycle[kl_coef][cycle][i] ** 2 + transitions_normalized_by_dim_cycle[kl_coef][cycle][i] ** 2)
            if distance < best_distance:
                best_distance = distance
                torch.save(model.state_dict(), f"{folder}/best_model_combined_distance.pth")
                print("New best model saved based on TVD + normalized average transition value.")
                index = i
                best_models_saved.append(model.state_dict())
        best_models.append(index)


    best_model_distance = float('inf')
    for i,n in enumerate(best_models):
        distance = np.sqrt(tvds_by_dim_cycle[kl_coef][i][n] ** 2 + transitions_normalized_by_dim_cycle[kl_coef][i][n] ** 2)
        if distance < best_model_distance:
            best_model_distance = distance
            best_model = best_models_saved[i]
            where_model = (i,n)
    # Save the best model  
    coordinates = where_model
    cycle = i
    epoch = n
    torch.save(best_model, f"{lat_folder}/best_model_combined_distance.pth")
    print(f"Best model saved based on TVD + normalized average transition value with cycle {i+1} and epoch {n+1}.")

In [ ]:
# Inference
import datetime
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_KL_TEST_at_2025-04-24 15:43:51.612862/model_at_1_with_kl_10/checkpoint_2"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")

for _ in range (0,5):
    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 300000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:300000, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    plt.figure(figsize=(20,8))
    plt.plot(pred_mean[0:300000,63],'r')
    plt.plot(actual_values[0:300000],'b')
    plt.grid(True)
    plt.title(f"Predictions vs Actual | Checkpoint 4(best)")
    plt.savefig(f'/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_KL_TEST_at_2025-04-24 15:43:51.612862/model_at_1_with_kl_10/prediction_vs_actual_{datetime.datetime.now()}.png')
    plt.show()

    # MODIFY THIS LINE FOR MODEL TESTING
    np.save(f'/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_KL_TEST_at_2025-04-24 15:43:51.612862/model_at_1_with_kl_10/predictions_best_checkpoint_and_cycle_Resnet_VAE_1.npy', pred_mean)

# NEW DATASET FOR FINETUNING + FINETUNING SCRIPT

In [ ]:
# CCDF Error bars

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns

CCDF_sum = 0

import datetime
import os

num_ccdfs = 10

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_2025-03-25 16:07:01.635719/checkpoint_4_promising"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")
    
for i in range(0, num_ccdfs):
    # Inference
    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 300000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    # Load the data; shape = (300000, 2, 75)
    real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
    predictions = pred_mean.reshape(300000, 1, 75)

    if (CCDF):
        real_data_1d = real_data[:, 1, 61]  # Now shape is (309700,)
        predictions_1d = predictions[:, 0, 61]  # shape (300000,)

        # Define bounds (assuming they apply to both datasets)
        upper_bound = 53.8 / 2.8935
        lower_bound = 1.75 / 2.8935

        # Function to calculate transition durations
        def calculate_transition_durations(y_values, upper_bound, lower_bound):
            times_between_transitions = []
            transition_start = None
            above_upper = False
            below_lower = False

            for i in range(1, len(y_values)):
                if y_values[i] < lower_bound:  
                    below_lower = True
                    above_upper = False
                elif y_values[i] > upper_bound:  
                    if below_lower and transition_start is not None:
                        times_between_transitions.append(i - transition_start)
                        transition_start = None  
                    above_upper = True
                    below_lower = False

                if below_lower and transition_start is None:
                    transition_start = i

            return times_between_transitions

        # Compute transition durations for real data
        real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

        # Compute transition durations for predictions data
        pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

        # Plot setup
        plt.figure(figsize=(10, 6))

        # === REAL DATA CCDF AND FIT ===
        if len(real_durations) == 0:
            print("No transitions detected in real data with current bounds!")
        else:
            real_data_sorted = np.sort(real_durations)
            x_line_real = np.linspace(min(real_data_sorted), max(real_data_sorted), 100)
            exponential_fit_real = 1/np.mean(real_data_sorted)
            y_values_real = exponential_fit_real*x_line_real
            plt.plot(x_line_real, y_values_real, 'b-', label=f'Real Exp Fit (slope={exponential_fit_real:.4f})', linewidth=2)

        # === PREDICTIONS CCDF AND FIT ===
        if len(pred_durations) == 0:
            print("No transitions detected in predictions with current bounds!")
        else:
            pred_data_sorted = np.sort(pred_durations)
            x_line_pred = np.linspace(min(pred_data_sorted), max(pred_data_sorted), 100)
            exponential_fit_pred = 1/np.mean(pred_data_sorted)
            y_values_pred = exponential_fit_pred*x_line_pred
            plt.plot(x_line_pred, y_values_pred, 'r-', label=f'Pred Exp Fit (slope={exponential_fit_pred:.4f})', linewidth=2)

        # Plot labels and formatting
        plt.xlabel('Time Duration (Steps)')
        plt.ylabel('CCDF')
        plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
        plt.yscale("linear")  # y-axis log scale
        plt.xscale("linear")  # x-axis linear scale
        plt.grid()
        plt.legend()
        plt.tight_layout()
        save_path = os.path.join(folder, "CCDF")
        save_path = os.path.join(save_path, "CCDF_plot")
        plt.savefig(save_path)
        plt.show()

CCDF_avg = CCDF_sum/num_ccdfs
print(CCDF_avg)

In [ ]:
# Init pack

import numpy as np
import h5py
import numpy as np
from scipy.integrate import solve_ivp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
import torch
import matplotlib.pyplot as plt

# Pre-processing
psi = np.load('/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy')

lead = 1

trainN = 200000
valN = 50000
index = 63

psi = psi[:,1,:]

# After normalizing the data but before creating tensors
upper_bound = 53.8 / 2.8935
lower_bound = 7.41

# Create a mask for values within bounds
psi_filtered = psi[:,63]

transitions = []
above_upper = False  # Flag to track if we are above upper bound
time_to_transition = []
time_taken = 0
time_to_transition_two = []

transitions_two = []
below_lower = False  # Flag to track if we are below lower bound

time_taken = 0
for i in range(1, len(psi_filtered)):
    time_taken += 1


    if psi_filtered[i] > upper_bound:
        above_upper = True
        

    elif above_upper and psi_filtered[i] < lower_bound:

        time_to_transition_two.append(time_taken)
        transitions.append(i)
        time_taken = 0
        above_upper = False  # Reset flag after transition


    if psi_filtered[i] < lower_bound:
        below_lower = True

    elif below_lower and psi_filtered[i] > upper_bound:
        transitions_two.append(i)


        time_to_transition.append(time_taken)
        time_taken = 0
        below_lower = False  # Reset flag after transition

for val in transitions:
    plt.axvline(x = val, color = 'r', label = 'axvline - full height')
for val in transitions_two:
    plt.axvline(x = val, color = 'b', label = 'axvline - full height')
plt.show()

print("Number of transitions from below lower to above upper bound:", len(transitions_two))
print("Number of transitions from above upper to below lower bound:", len(transitions))
transitions.extend(transitions_two)
transitions.sort()

print(len(transitions), transitions)

hit_bound = False
# THE STATE DEPENDS ON THE INITIAL CONDITION
weak_to_strong = True
strong_to_weak = False

series = []
for transition in transitions:
    index = transition
    if weak_to_strong:
        while not hit_bound:
            if psi_filtered[index] < lower_bound:
                trans = psi[index-10:transition+5,:]
                series.append(trans)
                hit_bound = True
            else:
                index -= 1
        weak_to_strong = False
        strong_to_weak = True
        hit_bound = False
    elif strong_to_weak:
        while not hit_bound:
            if psi_filtered[index] > upper_bound:
                trans = psi[index-10:transition+5,:]
                series.append(trans)
                hit_bound = True
            else:
                index -= 1
        weak_to_strong = True
        strong_to_weak = False
        hit_bound = False

print(psi.shape)
mean_psi = np.mean(psi, axis=0, keepdims=True)
std_psi = np.std(psi, axis=0, keepdims=True)
psi = (psi - mean_psi) / std_psi

psi_train_input = torch.tensor(psi[0:trainN,:])
psi_train_label =  torch.tensor(psi[lead:trainN+lead,:])

psi_val_input = torch.tensor(psi[trainN:trainN+valN,:])
psi_val_label =  torch.tensor(psi[trainN+lead:trainN+valN+lead,:])

len_series = []
normalized_series = []
for s in series:
    print(len(s))
    s = (s - mean_psi) / std_psi
    s = torch.tensor(s)
    len_series.append(len(s))
    normalized_series.append(s)
    plt.plot(s[:,63])

plt.show()
print(sum(len_series)/len(len_series))

# Total Variation Distance
def total_variation_distance(p,q):
	p = np.array(p)
	q = np.array(q)
	return 0.5 * np.sum(np.abs(p-q))

In [ ]:
# Training

import datetime
import os
import math

def set_model_finetune_mode():
    # Encoder
    model.encoder.fc1.weight.requires_grad = True
    model.encoder.fc2.weight.requires_grad = True
    model.encoder.fc3.weight.requires_grad = True
    model.encoder.fc4.weight.requires_grad = False
    model.encoder.fc5.weight.requires_grad = False
    model.encoder.fc6.weight.requires_grad = False
    model.encoder.fc_mu.weight.requires_grad = False
    model.encoder.fc_logvar.weight.requires_grad = False

    # Decoder
    model.decoder.fc1.weight.requires_grad = False
    model.decoder.fc2.weight.requires_grad = False
    model.decoder.fc3.weight.requires_grad = False
    model.decoder.fc4.weight.requires_grad = True
    model.decoder.fc5.weight.requires_grad = True
    model.decoder.fc6.weight.requires_grad = True

def set_model_eval_mode():
    # Encoder
    model.encoder.fc1.weight.requires_grad = True
    model.encoder.fc2.weight.requires_grad = True
    model.encoder.fc3.weight.requires_grad = True
    model.encoder.fc4.weight.requires_grad = True
    model.encoder.fc5.weight.requires_grad = True
    model.encoder.fc6.weight.requires_grad = True
    model.encoder.fc_mu.weight.requires_grad = True
    model.encoder.fc_logvar.weight.requires_grad = True

    # Decoder
    model.decoder.fc1.weight.requires_grad = True
    model.decoder.fc2.weight.requires_grad = True
    model.decoder.fc3.weight.requires_grad = True
    model.decoder.fc4.weight.requires_grad = True
    model.decoder.fc5.weight.requires_grad = True
    model.decoder.fc6.weight.requires_grad = True

latent_dims = [1024]
num_cycles = 1

tvds_by_dim_cycle = {dim: {cycle: [] for cycle in range(3)} for dim in latent_dims}
expo_fits_by_dim_cycle = {dim: {cycle: [] for cycle in range(3)} for dim in latent_dims}

for latent_dim in latent_dims:
    real_data = psi
    lat_folder = f"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_lat{latent_dim}_at_{datetime.datetime.now()}"
    os.makedirs(lat_folder)
    for cycle in range(0,num_cycles):

        # Initialize the model, optimizer, and loss function
        model = ConditionalVAE(latent_dim, output_dim, condition_dim)
        model = model.cuda()
        optimizer = optim.Adam(model.parameters(), lr=0.00001)
        num_epochs = 100

        model_weights_path = r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_KL_TEST_at_2025-04-24 15:43:51.612862/model_at_1_with_kl_10/checkpoint_2"

        if os.path.exists(model_weights_path):
            model.load_state_dict(torch.load(model_weights_path))
            print(f"Model weights loaded from {model_weights_path}.")
        else:
            print("can't find path")

        folder = f"{lat_folder}/model_at_{cycle}"
        os.makedirs(folder)
        subfolders = ['timeseries', 'expo_fit', '2D']

        # Create each subdirectory inside the main folder
        for subfolder in subfolders:
            path = os.path.join(folder, subfolder)
            os.mkdir(path)
            print(f"Created subfolder: {path}")

        # from torchsummary import summary

        # summary(model, input_size = [(128, 1, 75), (128, 75)])
        # Train the model
        
        for epoch in range(num_epochs):
            for i,s in enumerate(normalized_series):
                
                # TRAINING
                
                model.train()
                s_input = s[:-1,:]
                s_label = s[1:,:]

                optimizer.zero_grad()
                
                # Forward pass
                output, mu, logvar = model(s_label.float().cuda(), s_input.float().cuda())
                
                # Reconstruction loss
                reconstruction_loss = F.mse_loss(output, s_label.float().cuda(), reduction="sum")

                kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
                
                # Total loss
                loss = reconstruction_loss + 25 * kl_loss

                loss.backward()
                optimizer.step()
                
                # VALIDATION
                
                model.eval()
                s_input = s[:-1,:]
                s_label = s[1:,:]
                
                # Forward pass
                output, mu, logvar = model(s_label.float().cuda(), s_input.float().cuda())
                
                # Reconstruction loss
                reconstruction_val_loss = F.mse_loss(output, s_label.float().cuda(), reduction="sum")

                val_kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
                
                # Total loss
                val_loss = reconstruction_val_loss + 25 * kl_loss
                
            print(f'''Epoch {epoch+1}, transition {i+1},
                    Reconstrunction Loss: {reconstruction_loss}
                    KL Divergence Loss: {kl_loss}
                    Validation Reconstrunction Loss: {reconstruction_val_loss}
                    Validation KL Divergence Loss: {val_kl_loss}''')
            # Inference

            initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
            time_step = 30000
            z = torch.zeros([1,latent_dim])
            num_ens = 1
            pred = np.zeros ([time_step, 75, num_ens])

            for k in range (0, time_step):

                for ens in range (0, num_ens):
                    if (k ==0):

                        z = torch.randn_like(z)
                        print(z.shape, initial_cond.shape)
                        y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                        pred[k,:,ens] = y
                        y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                        initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

                    else:
                        select_ens = np.random.randint(0,num_ens,1)
                        z = torch.randn_like(z)
                        y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                        pred[k,:, ens] = y
                        y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                        initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            # Denormalize final preds
            print(std_psi[:, 63])
            pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

            # Denormalize test labels
            actual_values = psi_train_label[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
            print(actual_values)

            plt.figure(figsize=(16, 6))
            plt.plot(pred_mean[0:time_step,63],'r')
            plt.plot(actual_values[0:time_step],'b')
            plt.title(f"Predictions vs Actual | Batch size of {batch_size}")
            plt.savefig(f'{folder}/prediction_epoch_{epoch+1}.png')
            plt.show()

            predictions_1d = pred_mean[:, 63]  # shape (300000,)
            real_data_1d = real_data[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
            # Define bounds (assuming they apply to both datasets)
            upper_bound = 53.8 / 2.8935
            lower_bound = 7.41

            # Function to calculate transition durations
            def calculate_transition_durations(y_values, upper_bound, lower_bound):
                times_between_transitions = []
                transition_start = None
                above_upper = False
                below_lower = False

                for i in range(1, len(y_values)):
                    if y_values[i] <= lower_bound:  
                        below_lower = True
                        above_upper = False
                    elif y_values[i] >= upper_bound:  
                        if below_lower and transition_start is not None:
                            times_between_transitions.append(i - transition_start)
                            transition_start = None  
                        above_upper = True
                        below_lower = False

                    if below_lower and transition_start is None:
                        transition_start = i

                return times_between_transitions

            # Compute transition durations for real data
            real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

            # Compute transition durations for predictions data
            pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

            # Plot setup
            plt.figure(figsize=(10, 6))

            # === REAL DATA CCDF AND FIT ===
            if len(real_durations) == 0:
                print("No transitions detected in real data with current bounds!")
            else:
                real_data_sorted = np.sort(real_durations)
                x_line_real = np.linspace(min(real_data_sorted), max(real_data_sorted), 100)
                exponential_fit_real = 1/np.mean(real_data_sorted)
                y_values_real = exponential_fit_real*x_line_real
                plt.plot(x_line_real, y_values_real, 'b-', label=f'Real Exp Fit (slope={exponential_fit_real:.4f})', linewidth=2)

            # === PREDICTIONS CCDF AND FIT ===
            exponential_fit_pred = 0
            if len(pred_durations) == 0:
                print("No transitions detected in predictions with current bounds!")
            else:
                pred_data_sorted = np.sort(pred_durations)
                x_line_pred = np.linspace(min(pred_data_sorted), max(pred_data_sorted), 100)
                exponential_fit_pred = 1/np.mean(pred_data_sorted)
                y_values_pred = exponential_fit_pred*x_line_pred
                plt.plot(x_line_pred, y_values_pred, 'r-', label=f'Pred Exp Fit (slope={exponential_fit_pred:.4f})', linewidth=2)

            # Plot labels and formatting
            plt.xlabel('Time Duration (Steps)')
            plt.ylabel('Exponential Fit')
            plt.title('Exponential Fit of Time Between B->A and A->B Transitions (Exponential Fit)')
            plt.yscale("linear")  # y-axis log scale
            plt.xscale("linear")  # x-axis linear scale
            plt.grid()
            plt.legend()
            plt.tight_layout()
            save_path = os.path.join(folder, "expo_fit")
            save_path = os.path.join(save_path, f"expo_fit_plot_{epoch}.png")
            plt.savefig(save_path)
            plt.show()

            if pred_mean[:, 63].max() <= 100 or pred_mean[:, 63].min() >= -100:
                pred_hist, _ = np.histogram(pred_mean, bins=50, density=True)
                actual_hist, _ = np.histogram(actual_values,bins=50,density=True)

                tvd = total_variation_distance(pred_hist,actual_hist)

                save_path = folder
                save_path = os.path.join(folder, "2D")
                save_path = os.path.join(save_path, f"2D_plot_{epoch}.png")
                plt.plot(tvd, exponential_fit_pred, "ro", label=f"Point with {math.sqrt(tvd**2+(exponential_fit_pred-exponential_fit_real)**2)} distance from the optimal")
                plt.xlabel("Total Variation Distance")
                plt.ylabel("Exponential Fit")
                plt.title("2D Space of TVD and Exponential Fit")
                plt.xlim(0,1)
                plt.ylim(0,exponential_fit_real*2)
                plt.axhline(exponential_fit_real)
                plt.grid()
                plt.legend()
                plt.tight_layout()
                plt.savefig(save_path)
                plt.show()
            else:
                print("Weights experienced explosion!")

            tvds_by_dim_cycle[latent_dim][cycle].append(tvd)
            expo_fits_by_dim_cycle[latent_dim][cycle].append(exponential_fit_pred)

            if cycle == 2 and epoch == num_epochs - 1:  # Last epoch of last cycle for this dimension
                # Plot TVDs with cycles overlapped
                plt.figure(figsize=(10, 6))
                for c in range(3):
                    plt.plot(tvds_by_dim_cycle[latent_dim][c], 'o-', label=f'Cycle {c}')
                
                plt.xlabel('Epoch within Cycle')
                plt.ylabel('Total Variation Distance')
                plt.title(f'TVD Progress (Dim={latent_dim})')
                plt.grid(True)
                plt.legend()
                plt.savefig(f'{lat_folder}/tvd_dim_{latent_dim}_all_cycles.png')
                plt.close()
                
                # Plot Exponential Fits with cycles overlapped
                plt.figure(figsize=(10, 6))
                for c in range(3):
                    plt.plot(expo_fits_by_dim_cycle[latent_dim][c], 'o-', label=f'Cycle {c}')
                
                plt.axhline(y=exponential_fit_real, color='r', linestyle='--', label='Real Data')
                plt.xlabel('Epoch within Cycle')
                plt.ylabel('Exponential Fit Value')
                plt.title(f'Exponential Fit Progress (Dim={latent_dim})')
                plt.grid(True)
                plt.legend()
                plt.savefig(f'{lat_folder}/expo_fit_dim_{latent_dim}_all_cycles.png')
                plt.close()
                
                # If this is the last latent dimension, create master graphs with all dimensions
                if latent_dim == latent_dims[-1]:
                    # Create a master folder for overlapping graphs
                    master_folder = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/summary"
                    os.makedirs(master_folder, exist_ok=True)
                    
                    # Plot best TVD for each dimension (from last epoch of last cycle)
                    plt.figure(figsize=(10, 6))
                    best_tvds = [tvds_by_dim_cycle[dim][2][-1] for dim in latent_dims]  # Last value from last cycle
                    plt.bar(range(len(latent_dims)), best_tvds, tick_label=latent_dims)
                    plt.xlabel('Latent Dimension')
                    plt.ylabel('Final TVD Value')
                    plt.title('Best TVD Across Latent Dimensions')
                    plt.grid(axis='y')
                    plt.savefig(f'{master_folder}/best_tvd_comparison.png')
                    plt.close()
                    
                    # Plot best exponential fits for each dimension
                    plt.figure(figsize=(10, 6))
                    best_expos = [expo_fits_by_dim_cycle[dim][2][-1] for dim in latent_dims]  # Last value from last cycle
                    plt.bar(range(len(latent_dims)), best_expos, tick_label=latent_dims)
                    plt.axhline(y=exponential_fit_real, color='r', linestyle='--', label='Real Data')
                    plt.xlabel('Latent Dimension')
                    plt.ylabel('Final Exponential Fit Value')
                    plt.title('Best Exponential Fit Across Latent Dimensions')
                    plt.grid(axis='y')
                    plt.legend()
                    plt.savefig(f'{master_folder}/best_expo_fit_comparison.png')
                    plt.close()

            torch.save(model.state_dict(), f"{folder}/checkpoint_{epoch+1}")
            print(f"Model weights saved to {folder} with point {epoch+1}.")

In [ ]:
#FLAGS to determine testing
plot_data = 1
#what level do you want to plot
level = 63
CCDF = 1
Bi_modal_distribution = 1
single_step_profiles = 1
#for the single_step_profiles
NUM_SAMPLES = 5
#what weights do you want to use?
MODEL_PATH = r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_lat1024_at_2025-04-11 12:46:39.526973/model_at_0/checkpoint_3"
LEVEL = 63

import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

# Load the data; shape = (300000, 2, 75)
real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
predictions = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_lat1024_at_2025-04-11 12:46:39.526973/predictions_best_checkpoint_Finetuned_Resnet_VAE_1.npy")

#reshape the predictions so that it matches the real_data shape
predictions = predictions.reshape(300000, 1, 75)
print(predictions.shape)
print(real_data.shape)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S.%f")
folder = f"testing_at_{timestamp}"
os.mkdir(folder)
subfolders = ['timeseries', 'CCDF', 'bi_modal_distribution', 'single_step_profiles']
# Create each subdirectory inside the main folder
for subfolder in subfolders:
    path = os.path.join(folder, subfolder)
    os.mkdir(path)
    print(f"Created subfolder: {path}")
SAVE_DIR = os.path.join(folder, "single_step_profiles")

# LONG TRAINING RUN 

In [ ]:
# Training

import datetime
import os
import math

def normalize_transition_time(slope_value, delta, transition_real):
    normalized = 1 - np.exp(-np.abs((slope_value - transition_real)) / delta)
    return normalized

def total_variation_distance(p, q):
    p = np.array(p)
    q = np.array(q)
    return 0.5 * np.sum(np.abs(p - q))

def calculate_transition_durations(y_values, upper_bound, lower_bound):
    times_between_transitions = []
    transition_start = None
    above_upper = False
    below_lower = False
    for i in range(1, len(y_values)):
        if y_values[i] < lower_bound:
            below_lower = True
            above_upper = False
        elif y_values[i] > upper_bound:
            if below_lower and transition_start is not None:
                times_between_transitions.append(i - transition_start)
                transition_start = None
            above_upper = True
            below_lower = False
        if below_lower and transition_start is None:
            transition_start = i
    return times_between_transitions

latent_dims = [1024]
latent_dim = 1024
kl_coefficients = [1]
num_cycles = 1
test_epochs = [5, 10, 20, 50, 100, 200, 500, 1000, 2000, 3000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
upper_bound = 53.8 / 2.8935
lower_bound = 7.41
level = 63

best_distance = float('inf')
tvd_list = []
transition_list = []
transition_list_unormalized = []

real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
real_data_1d = real_data[:, 1, level]
real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)
real_data_sorted = np.sort(real_durations)
transition_real = np.mean(real_data_sorted)
actual_hist, bin_edges = np.histogram(real_data[:, 1, level], bins=50, density=True)
print(f"Reference Real Data average_transition_time: {transition_real}")

tvds_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}
transitions_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}
transitions_normalized_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}
exp_by_dim_cycle = {kl: {cycle: [] for cycle in range(num_cycles)} for kl in kl_coefficients}

lat_folder = f"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_KL_TEST_at_{datetime.datetime.now()}"
os.makedirs(lat_folder)

for kl_coef in kl_coefficients:
    print(f"USING KL COEF OF {kl_coef}")
    best_models = []
    best_models_saved = []
    for cycle in range(0,num_cycles):
        tvd_cycle_list = []
        transition_cycle_list = []
        transition_cycle_list_unormalized = []
        
        # Initialize the model, optimizer, and loss function
        model = ConditionalVAE(latent_dim, output_dim, condition_dim)
        model = model.cuda()
        optimizer = optim.Adam(model.parameters(), lr=0.0001)
        num_epochs = 10100

        folder = f"{lat_folder}/model_at_{cycle}_with_kl_{kl_coef}"
        os.makedirs(folder)
        subfolders = ['timeseries', 'expo_fit', '2D', 'summary']

        # Create each subdirectory inside the main folder
        for subfolder in subfolders:
            path = os.path.join(folder, subfolder)
            os.mkdir(path)
            print(f"Created subfolder: {path}")

        # from torchsummary import summary

        # summary(model, input_size = [(128, 1, 75), (128, 75)])
        # Train the model
        for epoch in range(num_epochs):
            model.train()
            for batch in range (0, trainN, batch_size):

                input_batch = psi_train_input[batch:batch + batch_size,:]
                label_batch = psi_train_label[batch:batch + batch_size,:]

                optimizer.zero_grad()

                output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())

                # Reconstruction loss
                reconstruction_loss = F.mse_loss(output, label_batch.float().cuda(), reduction="sum")
                # KL divergence loss
                kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
                
                # Total loss
                loss = reconstruction_loss + kl_coef * kl_loss

                loss.backward()
                optimizer.step()

            print(f'''Epoch {epoch+1}, 
                Reconstrunction Loss: {reconstruction_loss},
                KL Divergence Loss: {kl_loss}''')

            # Validation Loss
            for batch in range (0, valN, batch_size):

                input_batch = psi_val_input[batch:batch + batch_size,:]
                label_batch = psi_val_label[batch:batch + batch_size,:]
                
                output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())
                reconstruction_loss = F.mse_loss(output, label_batch.float().cuda(), reduction="sum")
                kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
                val_loss = reconstruction_loss + kl_coef * kl_loss # Experiment HIGHER coefficients
                # Print both reconstruction_loss and kl_loss

            print(f'''
                Validation Reconstrunction Loss: {reconstruction_loss},
                Validation KL Divergence Loss: {kl_loss}''')
            
            # Inference

            if epoch-1 in test_epochs:
                initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
                time_step = 300000
                z = torch.zeros([1,latent_dim])
                num_ens = 1
                pred = np.zeros ([time_step, 75, num_ens])

                for k in range (0, time_step):

                    for ens in range (0, num_ens):
                        if (k ==0):

                            z = torch.randn_like(z)
                            print(z.shape, initial_cond.shape)
                            y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                            pred[k,:,ens] = y
                            y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                            initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

                        else:
                            select_ens = np.random.randint(0,num_ens,1)
                            z = torch.randn_like(z)
                            y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                            pred[k,:, ens] = y
                            y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                            initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

                # Denormalize final preds
                print(std_psi[:, 63])
                pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

                # Denormalize test labels
                actual_values = psi_train_label[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
                print(actual_values)
                
                real_data_1d = real_data[:30000, 0, 63]  # Now shape is (309700,)
                predictions_1d = pred_mean[:, 63]  # shape (300000,)

                plt.figure(figsize=(20,8))
                plt.plot(pred_mean[0:30000,63],'r')
                plt.plot(real_data[0:30000, 0, 63])
                plt.grid(True)
                plt.title(f"Predictions vs Actual | Epoch {epoch}")
                save_path = os.path.join(folder, "timeseries")
                save_path = os.path.join(save_path, f"timeseries_plot_{epoch}.png")
                plt.savefig(save_path)
                plt.xlabel('Time Step')
                plt.ylabel('Zonal Wind Value')
                plt.legend(['Predictions', 'Actual'])
                plt.tight_layout()
                plt.show()
                
                # Function to calculate transition durations
                def calculate_transition_durations(y_values, upper_bound, lower_bound):
                    times_between_transitions = []
                    transition_start = None
                    above_upper = False
                    below_lower = False

                    for i in range(1, len(y_values)):
                        if y_values[i] < lower_bound:  
                            below_lower = True
                            above_upper = False
                        elif y_values[i] > upper_bound:  
                            if below_lower and transition_start is not None:
                                times_between_transitions.append(i - transition_start)
                                transition_start = None  
                            above_upper = True
                            below_lower = False

                        if below_lower and transition_start is None:
                            transition_start = i

                    return times_between_transitions
                
                predictions_1d = pred_mean[:, level, 0]
                pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)
                print(pred_durations)
                pred_hist, _ = np.histogram(predictions_1d, bins=bin_edges, density=True)

                # === PREDICTIONS CCDF AND FIT ===
                exponential_fit_pred = 0

                # if len(pred_durations) > 0:
                transition_pred = np.mean(pred_durations)
                transition_cycle_list.append(normalize_transition_time(transition_pred, 1000, transition_real))
                transition_cycle_list_unormalized.append(transition_pred)
                transitions_by_dim_cycle[kl_coef][cycle].append(transition_pred)
                transitions_normalized_by_dim_cycle[kl_coef][cycle].append(normalize_transition_time(transition_pred, 1000, transition_real))

                tvd = total_variation_distance(pred_hist, actual_hist)
                tvd_cycle_list.append(tvd)
                tvds_by_dim_cycle[kl_coef][cycle].append(tvd)

                distance = np.sqrt(tvd ** 2 + (normalize_transition_time(transition_pred, 1000, transition_real)) ** 2)

                print(f"Epoch {epoch+1}: TVD = {tvd:.6f}, Transition_Difference = {transition_pred-transition_real:.6f}, Combined Distance = {distance:.6f}")
                torch.save(model.state_dict(), f"{folder}/model_at_epoch{epoch}")

                if distance < best_distance:
                    best_distance = distance
                    torch.save(model.state_dict(), f"{folder}/best_model_combined_distance.pth")
                    print("New best model saved based on TVD + transition difference distance.")
                # else:
                #     print("No transitions detected in predictions.")
                #     transitions_by_dim_cycle[kl_coef][cycle].append(np.nan)
                #     tvds_by_dim_cycle[kl_coef][cycle].append(np.nan)
                #     tvd_cycle_list.append(np.nan)
                #     transition_cycle_list.append(np.nan)
                #     transition_cycle_list_unormalized.append(np.nan)


                # Plot labels and formatting

                x_line_real = np.linspace(min(real_data_sorted), max(real_data_sorted), 100)
                exponential_fit_real = 1/np.mean(real_data_sorted)
                y_values_real = exponential_fit_real*x_line_real
                plt.plot(x_line_real, y_values_real, 'b-', label=f'Real Exp Fit (slope={exponential_fit_real:.4f})', linewidth=2)

                if len(pred_durations) > 0:
                    x_line_pred = np.linspace(min(pred_durations), max(pred_durations), 100)
                    exponential_fit_pred = 1/np.mean(pred_durations)
                    y_values_pred = exponential_fit_pred*x_line_pred
                    plt.plot(x_line_pred, y_values_pred, 'r-', label=f'Pred Exp Fit (slope={exponential_fit_pred:.4f})', linewidth=2)
                    exp_by_dim_cycle[kl_coef][cycle].append(exponential_fit_pred)

                plt.xlabel('Time Duration (Steps)')
                plt.ylabel('Exponential Fit')
                plt.title('Exponential Fit of Time Between B->A and A->B Transitions (Exponential Fit)')
                plt.yscale("linear")  # y-axis log scale
                plt.xscale("linear")  # x-axis linear scale
                plt.grid()
                plt.legend()
                plt.tight_layout()
                save_path = os.path.join(folder, "expo_fit")
                save_path = os.path.join(save_path, f"expo_fit_plot_{epoch}.png")
                plt.savefig(save_path)
                plt.show()

                plt.figure(figsize=(12, 6))
                plt.plot(tvd, transition_pred, 'o-', label=f'Cycle {cycle}')
                plt.xlabel("Total Variation Distance")
                plt.ylabel("Average Transition Time")
                plt.axhline(y=transition_real, color='r', linestyle='--', label='Real Data')
                plt.ylim(0.1,2000)
                plt.xlim(0, 1)
                plt.title("TVD vs. Avg Transition Time per Epoch")
                plt.grid(True)
                save_path = os.path.join(folder, "2D")
                save_path = os.path.join(save_path, f"2D_plot_{epoch}.png")
                plt.savefig(save_path)
                plt.show()
                
                if epoch == num_epochs - 1:  # Last epoch of last cycle for this dimension
                    # Plot TVDs with cycles overlapped
                    plt.figure(figsize=(10, 6))
                    for c in range(num_cycles):
                        if c in tvds_by_dim_cycle[kl_coef]:
                            plt.plot(tvds_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                        else:
                            print(f"Skipping cycle {c} as it is not in tvds_by_dim_cycle.")
                    
                    plt.xlabel('Epoch within Cycle')
                    plt.ylabel('Total Variation Distance')
                    plt.title(f'TVD Progress (KL Coefficient={kl_coef})')
                    plt.grid(True)
                    plt.legend()
                    save_path = os.path.join(folder, "summary")
                    save_path = os.path.join(save_path, f"tvd_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()
                    for c in range(num_cycles):
                        if c in transitions_by_dim_cycle[kl_coef]:
                            plt.plot(transitions_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                        else:
                            print(f"Skipping cycle {c} as it is not in transitions_by_dim_cycle.")
                    plt.figure(figsize=(10, 6))
                    for c in range(num_cycles):
                        plt.plot(transitions_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                    
                    plt.axhline(y=transition_real, color='r', linestyle='--', label='Real Data')
                    plt.xlabel('Epoch within Cycle')
                    plt.ylabel('Average Transition Value')
                    plt.ylim(0.1,2000)
                    plt.title(f'Average Transition Progress (KL Coefficient={kl_coef})')
                    plt.grid(True)
                    plt.legend()
                    save_path = os.path.join(folder, "summary")
                    save_path = os.path.join(save_path, f"transition_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()

                    plt.figure(figsize=(10, 6))
                    for c in range(num_cycles):
                        plt.plot(exp_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                    
                    plt.axhline(y=exponential_fit_real, color='r', linestyle='--', label='Real Data')
                    plt.xlabel('Epoch within Cycle')
                    plt.ylabel('Exponential Fit Value')
                    plt.title(f'Exponential Fit Progress (KL Coefficient={kl_coef})')
                    plt.grid(True)
                    plt.legend()
                    save_path = os.path.join(folder, "summary")
                    save_path = os.path.join(save_path, f"exponential_fit_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()
                    
                    # Plot TVD and 1/average transition time per epoch
                    plt.figure(figsize=(12, 6))
                    plt.scatter(tvd_cycle_list, transition_cycle_list, c=range(len(tvd_cycle_list)), cmap='viridis')
                    plt.colorbar(label='Epoch')
                    plt.xlabel("Total Variation Distance")
                    plt.ylabel("Average Transition Time")
                    plt.title("TVD vs. Avg Transition Time per Epoch")
                    plt.grid(True)
                    save_path = os.path.join(folder, "summary")
                    save_path = os.path.join(save_path, f"tvd_vs_transition_plot_all_cycles.png")
                    plt.savefig(save_path)
                    plt.show()
                    
                    # If this is the last latent dimension, create master graphs with all dimensions
                    if kl_coef == kl_coefficients[-1] and cycle == num_cycles - 1:
                        # Create a master folder for overlapping graphs
                            
                            # Plot TVDs with cycles overlapped
                        plt.figure(figsize=(10, 6))
                        for c in range(num_cycles):
                            plt.plot(tvds_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                        
                        plt.xlabel('Epoch within Cycle')
                        plt.ylabel('Total Variation Distance')
                        plt.title(f'TVD Progress (KL Coefficient={kl_coef})')
                        plt.grid(True)
                        plt.legend()
                        save_path = os.path.join(lat_folder, f"tvd_plot_all_cycles.png")
                        plt.savefig(save_path)
                        plt.show()
                        
                        # Plot Exponential Fits with cycles overlapped
                        plt.figure(figsize=(10, 6))
                        for c in range(num_cycles):
                            plt.plot(transitions_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                        
                        plt.axhline(y=transition_real, color='r', linestyle='--', label='Real Data')
                        plt.xlabel('Epoch within Cycle')
                        plt.ylabel('Average Transition Value')
                        plt.ylim(0.1,2000)
                        plt.title(f'Average Transition Progress (KL Coefficient={kl_coef})')
                        plt.grid(True)
                        plt.legend()
                        save_path = os.path.join(lat_folder, f"transition_plot_all_cycles.png")
                        plt.savefig(save_path)
                        plt.show()

                        plt.figure(figsize=(10, 6))
                        for c in range(num_cycles):
                            plt.plot(exp_by_dim_cycle[kl_coef][c], 'o-', label=f'Cycle {c}')
                        
                        plt.axhline(y=exponential_fit_real, color='r', linestyle='--', label='Real Data')
                        plt.xlabel('Epoch within Cycle')
                        plt.ylabel('Exponential Fit Value')
                        plt.title(f'Exponential Fit Progress (KL Coefficient={kl_coef})')
                        plt.grid(True)
                        plt.legend()
                        save_path = os.path.join(lat_folder, f"exponential_fit_plot_all_cycles.png")
                        plt.savefig(save_path)
                        plt.show()
                        
                        # Plot TVD and 1/average transition time per epoch
                        plt.figure(figsize=(12, 6))
                        plt.scatter(tvd_cycle_list, transition_cycle_list, c=range(len(tvd_cycle_list)), cmap='viridis')
                        plt.colorbar(label='Epoch')
                        plt.xlabel("Total Variation Distance")
                        plt.ylabel("Average Transition Time")
                        plt.title("TVD vs. Avg Transition Time per Epoch")
                        plt.grid(True)
                        save_path = os.path.join(lat_folder, f"tvd_vs_transition_plot_all_cycles.png")
                        plt.savefig(save_path)
                        plt.show()

                torch.save(model.state_dict(), f"{folder}/checkpoint_{epoch+1}")
                print(f"Model weights saved to {folder} with point {epoch+1}.")
            
                tvd_list.append(tvd_cycle_list)
                
            for i in range(0, num_cycles):
                # Ensure the index exists in the lists
                if i < len(tvds_by_dim_cycle[kl_coef][cycle]) and i < len(transitions_normalized_by_dim_cycle[kl_coef][cycle]):
                    distance = np.sqrt(tvds_by_dim_cycle[kl_coef][cycle][i] ** 2 + transitions_normalized_by_dim_cycle[kl_coef][cycle][i] ** 2)
                    if distance < best_distance:
                        best_distance = distance
                        torch.save(model.state_dict(), f"{folder}/best_model_combined_distance.pth")
                        print("New best model saved based on TVD + normalized average transition value.")
                        index = i
                        best_models_saved.append(model.state_dict())
                else:
                    print(f"Skipping index {i} as it is out of range.")
            best_models.append(index)


        best_model_distance = float('inf')
        for i,n in enumerate(best_models):
            distance = np.sqrt(tvds_by_dim_cycle[kl_coef][i][n] ** 2 + transitions_normalized_by_dim_cycle[kl_coef][i][n] ** 2)
            if distance < best_model_distance:
                best_model_distance = distance
                best_model = best_models_saved[i]
                where_model = (i,n)
        # Save the best model  
        coordinates = where_model
        cycle = i
        epoch = n
        torch.save(best_model, f"{lat_folder}/best_model_combined_distance.pth")
        print(f"Best model saved based on TVD + normalized average transition value with cycle {i+1} and epoch {n+1}.")

# TESTING OF MODELS AND INFERENCES

In [ ]:
#FLAGS to determine testing
plot_data = 1
#what level do you want to plot
level = 63
CCDF = 1
Bi_modal_distribution = 1
single_step_profiles = 1
#for the single_step_profiles
NUM_SAMPLES = 5
#what weights do you want to use?
MODEL_PATH = r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_lat1024_at_2025-04-11 12:46:39.526973/model_at_0/checkpoint_3"
LEVEL = 63

import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

# Load the data; shape = (300000, 2, 75)
real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
predictions = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_lat1024_at_2025-04-11 12:46:39.526973/predictions_best_checkpoint_Finetuned_Resnet_VAE_1.npy")

#reshape the predictions so that it matches the real_data shape
predictions = predictions.reshape(300000, 1, 75)
print(predictions.shape)
print(real_data.shape)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S.%f")
folder = f"testing_at_{timestamp}"
os.mkdir(folder)
subfolders = ['timeseries', 'CCDF', 'bi_modal_distribution', 'single_step_profiles']
# Create each subdirectory inside the main folder
for subfolder in subfolders:
    path = os.path.join(folder, subfolder)
    os.mkdir(path)
    print(f"Created subfolder: {path}")
SAVE_DIR = os.path.join(folder, "single_step_profiles")

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns
import random

model = ConditionalVAE(latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_lat1024_at_2025-04-11 12:46:39.526973/model_at_0/checkpoint_3"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")
    
if (plot_data):
    #note that the value 300000 will have to change depending on the real and predictions data length
    u_profile_real = real_data[:300000, 1, level]  # Match time length with predictions
    u_profile_pred = predictions[:, 0, level]
    time_steps = np.arange(len(u_profile_pred))

    # === Plot ===
    plt.figure(figsize=(14, 6))

    plt.plot(time_steps, u_profile_real, label='Real Data', alpha=0.7)
    plt.plot(time_steps, u_profile_pred, label='Predictions', linestyle='--')


    # Labels, legend, and formatting
    plt.xlabel('Time step')
    plt.ylabel('U (m/s)')
    plt.title(f'Time Series of U at Vertical Level {level}')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    save_path = os.path.join(folder, "timeseries")
    save_path = os.path.join(save_path, "real_prediction_plot")
    plt.savefig(save_path)
    plt.show()

if (CCDF):
    real_data_1d = real_data[:, 1, 63]  # Now shape is (309700,)
    predictions_1d = predictions[:, 0, 63]  # shape (300000,)

    # Define bounds (assuming they apply to both datasets)
    upper_bound = 53.8 / 2.8935
    lower_bound = 1.75 / 2.8935

    # Function to calculate transition durations
    def calculate_transition_durations(y_values, upper_bound, lower_bound):
        times_between_transitions = []
        transition_start = None
        above_upper = False
        below_lower = False

        for i in range(1, len(y_values)):
            if y_values[i] < lower_bound:  
                below_lower = True
                above_upper = False
            elif y_values[i] > upper_bound:  
                if below_lower and transition_start is not None:
                    times_between_transitions.append(i - transition_start)
                    transition_start = None  
                above_upper = True
                below_lower = False

            if below_lower and transition_start is None:
                transition_start = i

        return times_between_transitions

    # Compute transition durations for real data
    real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

    # Compute transition durations for predictions data
    pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

    # Plot setup
    plt.figure(figsize=(10, 6))

    # === REAL DATA CCDF AND FIT ===
    if len(real_durations) == 0:
        print("No transitions detected in real data with current bounds!")
    else:
        real_data_sorted = np.sort(real_durations)
        x_line_real = np.linspace(min(real_data_sorted), max(real_data_sorted), 100)
        exponential_fit_real = 1/np.mean(real_data_sorted)
        y_values_real = exponential_fit_real*x_line_real
        plt.plot(x_line_real, y_values_real, 'b-', label=f'Real Exp Fit (slope={exponential_fit_real:.4f})', linewidth=2)

    # === PREDICTIONS CCDF AND FIT ===
    if len(pred_durations) == 0:
        print("No transitions detected in predictions with current bounds!")
    else:
        pred_data_sorted = np.sort(pred_durations)
        x_line_pred = np.linspace(min(pred_data_sorted), max(pred_data_sorted), 100)
        exponential_fit_pred = 1/np.mean(pred_data_sorted)
        y_values_pred = exponential_fit_pred*x_line_pred
        plt.plot(x_line_pred, y_values_pred, 'r-', label=f'Pred Exp Fit (slope={exponential_fit_pred:.4f})', linewidth=2)

    print(1/np.mean(real_data_sorted))
    print(1/np.mean(pred_data_sorted))
    # Plot labels and formatting
    plt.xlabel('Time Duration (Steps)')
    plt.ylabel('CCDF')
    plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
    plt.yscale("linear")  # y-axis log scale
    plt.xscale("linear")  # x-axis linear scale
    plt.grid()
    plt.legend()
    plt.tight_layout()
    save_path = os.path.join(folder, "CCDF")
    save_path = os.path.join(save_path, "CCDF_plot")
    plt.savefig(save_path)
    plt.show()
if (Bi_modal_distribution):
    zonal_wind_data_real = real_data[:, 1, 63]  # variable index 1 (e.g., zonal wind), level 60
    zonal_wind_data_predictions = predictions[:, 0, 63]  # variable index 0 (predictions), level 60

    print(f"Shape of zonal_wind_data_real: {zonal_wind_data_real.shape}")
    print(f"Shape of zonal_wind_data_predictions: {zonal_wind_data_predictions.shape}")

    # Plot the bimodal histogram
    plt.figure(figsize=(12, 6))

    # Create histograms (overlaid)
    sns.histplot(zonal_wind_data_real, bins=50, kde=True, color='black', alpha=0.6, element='step', label='Real Data')
    sns.histplot(zonal_wind_data_predictions, bins=50, kde=True, color='red', alpha=0.6, element='step', label='Predictions')

    # Customize plot labels and title
    plt.title('Distribution of Zonal Winds For Real Data and Predictions', fontsize=16)
    plt.xlabel('Zonal Wind (m/s)', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)

    # Add vertical lines at means
    plt.axvline(np.mean(zonal_wind_data_real), color='black', linestyle='--', label=f'Real Mean: {np.mean(zonal_wind_data_real):.2f}')
    plt.axvline(np.mean(zonal_wind_data_predictions), color='red', linestyle='--', label=f'Pred Mean: {np.mean(zonal_wind_data_predictions):.2f}')

    # Final plot settings
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    save_path = os.path.join(folder, "bi_modal_distribution")
    save_path = os.path.join(save_path, "bi_modal_distribution_plot")
    plt.savefig(save_path)
    plt.show()
if (single_step_profiles):
    # Ensure save directory exists
    os.makedirs(SAVE_DIR, exist_ok=True)

    # === Load model weights ===
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()

    # === Randomly sample time points from real data ===
    time_indices = random.sample(range(0, real_data.shape[0] - 2), NUM_SAMPLES)
    print(f"Randomly sampled time steps: {time_indices}")

    # === Time series visualization ===
    real_data_timeseries = real_data[:, 1, LEVEL]
    time_steps_all = np.arange(len(real_data_timeseries))

    plt.figure(figsize=(12, 6))
    plt.plot(time_steps_all, real_data_timeseries, label="Real Data at Level 61", color='blue')

    # Mark sample points
    for idx_num, idx in enumerate(time_indices):
        plt.axvline(x=idx, color='green', linestyle='--', linewidth=2)
    if len(time_indices) > 0:
        plt.axvline(x=time_indices[0], color='green', linestyle='--', linewidth=2, label='Sampled Points')

    plt.title("Real Data Time Series with Sampled Points Highlighted")
    plt.xlabel("Time Step")
    plt.ylabel("U (m/s) at Level 61")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    save_path = os.path.join(SAVE_DIR, "real_data_timeseries_with_samples.png")
    plt.savefig(save_path)
    plt.show()

    # === Iterate over each sampled time point ===
    for i, time_step in enumerate(time_indices):
        next_time_step = time_step + 1

        # === Real data: current and next ===
        real_current = real_data[time_step, 1, :]       
        real_next = real_data[next_time_step, 1, :]      

        # === Normalize real_current and make prediction for next step ===
        initial_cond = torch.reshape(torch.tensor(psi[time_step,:]), [1, 75])
        z = torch.zeros([1,latent_dim])
        num_ens = 1
        pred = np.zeros ([time_step, 75, num_ens])

        with torch.no_grad():
            z = torch.randn_like(z)
            print(z.shape, initial_cond.shape)
            y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()

        # === Denormalize predicted next ===
        pred_next_denorm = y.squeeze() * std_psi.squeeze() + mean_psi.squeeze()

        # === Extract U, Re(Psi), Im(Psi) components ===
        # U profiles
        U_current_real = real_current[51:74]
        U_next_real = real_next[51:74]
        U_next_pred = pred_next_denorm[51:74]

        # Re(Psi) profiles
        RePsi_current_real = real_current[0:24]
        RePsi_next_real = real_next[0:24]
        RePsi_next_pred = pred_next_denorm[0:24]

        # Im(Psi) profiles
        ImPsi_current_real = real_current[25:50]
        ImPsi_next_real = real_next[25:50]
        ImPsi_next_pred = pred_next_denorm[25:50]

        # === Differences ===
        U_diff_real = U_next_real - U_current_real
        U_diff_pred = U_next_pred - U_current_real

        RePsi_diff_real = RePsi_next_real - RePsi_current_real
        RePsi_diff_pred = RePsi_next_pred - RePsi_current_real

        ImPsi_diff_real = ImPsi_next_real - ImPsi_current_real
        ImPsi_diff_pred = ImPsi_next_pred - ImPsi_current_real

        # === Create a single figure with 3 rows (U, Re(Psi), Im(Psi)) ===
        fig, axes = plt.subplots(3, 2, figsize=(16, 18))  # 3 rows, 2 columns (Profile and Difference)

        z_levels_U = np.linspace(0, 70, 23)
        z_levels_RePsi = np.linspace(0, 70, 24)
        z_levels_ImPsi = np.linspace(0, 70, 25)

        # --- U ---
        axes[0, 0].plot(U_current_real, z_levels_U, 'x-', label="Real Current")
        axes[0, 0].plot(U_next_real, z_levels_U, 'd-', label="Real Next")
        axes[0, 0].plot(U_next_pred, z_levels_U, 's--', label="Predicted Next")
        axes[0, 0].set_title(f"U Profiles @ Step {time_step}")
        axes[0, 0].set_xlabel("U (m/s)")
        axes[0, 0].set_ylabel("Vertical Levels (km)")
        axes[0, 0].legend()

        axes[0, 1].plot(U_diff_real, z_levels_U, 'xb', label="Real Δ (Next - Current)")
        axes[0, 1].plot(U_diff_pred, z_levels_U, 'o--r', label="Pred Δ (Next - Current)")
        axes[0, 1].set_title("U Difference (Next - Current)")
        axes[0, 1].set_xlabel("ΔU (m/s)")
        axes[0, 1].legend()

        # --- Re(Psi) ---
        axes[1, 0].plot(RePsi_current_real, z_levels_RePsi, 'x-', label="Real Current")
        axes[1, 0].plot(RePsi_next_real, z_levels_RePsi, 'd-', label="Real Next")
        axes[1, 0].plot(RePsi_next_pred, z_levels_RePsi, 's--', label="Predicted Next")
        axes[1, 0].set_title(f"Re(Psi) Profiles @ Step {time_step}")
        axes[1, 0].set_xlabel("Re(Psi)")
        axes[1, 0].set_ylabel("Vertical Levels (km)")
        axes[1, 0].legend()

        axes[1, 1].plot(RePsi_diff_real, z_levels_RePsi, 'xb', label="Real Δ (Next - Current)")
        axes[1, 1].plot(RePsi_diff_pred, z_levels_RePsi, 'o--r', label="Pred Δ (Next - Current)")
        axes[1, 1].set_title("Re(Psi) Difference (Next - Current)")
        axes[1, 1].set_xlabel("ΔRe(Psi)")
        axes[1, 1].legend()

        # --- Im(Psi) ---
        axes[2, 0].plot(ImPsi_current_real, z_levels_ImPsi, 'x-', label="Real Current")
        axes[2, 0].plot(ImPsi_next_real, z_levels_ImPsi, 'd-', label="Real Next")
        axes[2, 0].plot(ImPsi_next_pred, z_levels_ImPsi, 's--', label="Predicted Next")
        axes[2, 0].set_title(f"Im(Psi) Profiles @ Step {time_step}")
        axes[2, 0].set_xlabel("Im(Psi)")
        axes[2, 0].set_ylabel("Vertical Levels (km)")
        axes[2, 0].legend()

        axes[2, 1].plot(ImPsi_diff_real, z_levels_ImPsi, 'xb', label="Real Δ (Next - Current)")
        axes[2, 1].plot(ImPsi_diff_pred, z_levels_ImPsi, 'o--r', label="Pred Δ (Next - Current)")
        axes[2, 1].set_title("Im(Psi) Difference (Next - Current)")
        axes[2, 1].set_xlabel("ΔIm(Psi)")
        axes[2, 1].legend()

        # === Finalize and Save ===
        plt.suptitle(f"Single Step Profile Comparisons at Time Step {time_step}", fontsize=18)
        plt.tight_layout(rect=[0, 0.03, 1, 0.97])

        save_path = os.path.join(SAVE_DIR, f"Profile_Summary_point_{time_step}.png")
        plt.savefig(save_path)
        plt.show()

        print(f"Combined plot saved for sampled point {time_step}")

    # Final debug
    print("Finished processing all sampled points.")
        # Debugging prints
    print(predictions.shape) 
    print(real_data.shape)

# EXPERIMENTAL (AVG CCDFs WITH ERROR BARS)

In [ ]:
# CCDF Error bars

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns

CCDF_sum = 0

import datetime
import os

num_ccdfs = 10

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_2025-03-25 16:07:01.635719/checkpoint_4_promising"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")
    
for i in range(0, num_ccdfs):
    # Inference
    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 300000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    # Load the data; shape = (300000, 2, 75)
    real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
    predictions = pred_mean.reshape(300000, 1, 75)

    if (CCDF):
        real_data_1d = real_data[:, 1, 61]  # Now shape is (309700,)
        predictions_1d = predictions[:, 0, 61]  # shape (300000,)

        # Define bounds (assuming they apply to both datasets)
        upper_bound = 53.8 / 2.8935
        lower_bound = 1.75 / 2.8935

        # Function to calculate transition durations
        def calculate_transition_durations(y_values, upper_bound, lower_bound):
            times_between_transitions = []
            transition_start = None
            above_upper = False
            below_lower = False

            for i in range(1, len(y_values)):
                if y_values[i] < lower_bound:  
                    below_lower = True
                    above_upper = False
                elif y_values[i] > upper_bound:  
                    if below_lower and transition_start is not None:
                        times_between_transitions.append(i - transition_start)
                        transition_start = None  
                    above_upper = True
                    below_lower = False

                if below_lower and transition_start is None:
                    transition_start = i

            return times_between_transitions

        # Compute transition durations for real data
        real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

        # Compute transition durations for predictions data
        pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

        # Plot setup
        plt.figure(figsize=(10, 6))

        # === REAL DATA CCDF AND FIT ===
        if len(real_durations) == 0:
            print("No transitions detected in real data with current bounds!")
        else:
            real_data_sorted = np.sort(real_durations)
            x_line_real = np.linspace(min(real_data_sorted), max(real_data_sorted), 100)
            exponential_fit_real = 1/np.mean(real_data_sorted)
            y_values_real = exponential_fit_real*x_line_real
            plt.plot(x_line_real, y_values_real, 'b-', label=f'Real Exp Fit (slope={exponential_fit_real:.4f})', linewidth=2)

        # === PREDICTIONS CCDF AND FIT ===
        if len(pred_durations) == 0:
            print("No transitions detected in predictions with current bounds!")
        else:
            pred_data_sorted = np.sort(pred_durations)
            x_line_pred = np.linspace(min(pred_data_sorted), max(pred_data_sorted), 100)
            exponential_fit_pred = 1/np.mean(pred_data_sorted)
            y_values_pred = exponential_fit_pred*x_line_pred
            plt.plot(x_line_pred, y_values_pred, 'r-', label=f'Pred Exp Fit (slope={exponential_fit_pred:.4f})', linewidth=2)

        # Plot labels and formatting
        plt.xlabel('Time Duration (Steps)')
        plt.ylabel('CCDF')
        plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
        plt.yscale("linear")  # y-axis log scale
        plt.xscale("linear")  # x-axis linear scale
        plt.grid()
        plt.legend()
        plt.tight_layout()
        save_path = os.path.join(folder, "CCDF")
        save_path = os.path.join(save_path, "CCDF_plot")
        plt.savefig(save_path)
        plt.show()

CCDF_avg = CCDF_sum/num_ccdfs
print(CCDF_avg)